# Chess-Bot
An implementation of DeepChess (paper in `docs/deepchess.pdf`) and its use in a chess engine
### Differences from the original DeepChess:
- Training of the Autoencoder uses BinaryCrossentropy as its loss function. It's unclear what is used in the paper, but I suspect MSE
- LeakyReLU instead of ReLU?

## Setup

In [1]:
%load_ext autoreload
%autoreload 2

import chessbot as cb

## Environment
Downloads, unpacks, processes, and splits training data  
Should this break, go to http://ccrl.chessdom.com/ccrl/4040/games.html and replace the variable in `chessbot/environment.py` with the updated no-comments game link

In [2]:
cb.environment.create()

Dataset archive file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn.7z
Dataset file found at /home/wumphlett/personal/Chess-Bot/data/CCRL-4040.[1505357].pgn


Processing:   0%|          | 0/1505357 [00:00<?, ?it/s]

Sampling:   0%|          | 0/2 [00:00<?, ?it/s]

Done


## Pos2Vec
The Pos2Vec dataset consists of 1mil positions each from white won and black won games  
Pos2Vec is trained using a greedy layer-by-layer strategy of an unsupervised autoencoder

In [3]:
train, val = cb.dataset.pos2vec_dataset()

In [4]:
pos2vec = cb.model.train_pos2vec(train, val)

Training layer 1/4
Epoch 1/200
125000/125000 [==============================] - 241s 2ms/step - loss: 0.0454 - binary_accuracy: 0.9511 - val_loss: 0.0235 - val_binary_accuracy: 0.9750 - lr: 0.0050
Epoch 2/200
125000/125000 [==============================] - 244s 2ms/step - loss: 0.0224 - binary_accuracy: 0.9760 - val_loss: 0.0215 - val_binary_accuracy: 0.9765 - lr: 0.0049
Epoch 3/200
125000/125000 [==============================] - 242s 2ms/step - loss: 0.0208 - binary_accuracy: 0.9768 - val_loss: 0.0202 - val_binary_accuracy: 0.9776 - lr: 0.0048
Epoch 4/200
125000/125000 [==============================] - 243s 2ms/step - loss: 0.0197 - binary_accuracy: 0.9776 - val_loss: 0.0192 - val_binary_accuracy: 0.9786 - lr: 0.0047
Epoch 5/200
125000/125000 [==============================] - 246s 2ms/step - loss: 0.0188 - binary_accuracy: 0.9787 - val_loss: 0.0184 - val_binary_accuracy: 0.9797 - lr: 0.0046
Epoch 6/200
125000/125000 [==============================] - 247s 2ms/step - loss: 0.0180 -

KeyboardInterrupt: 

## DeepChess
The DeepChess dataset consists of 1mil position pairs randomly ordered  
DeepChess is trained using a shared Pos2Vec instance to perform feature extraction then fed into another neural net to determine which position is best

In [ ]:
train, val = cb.dataset.deepchess_dataset()

In [ ]:
deepchess = cb.model.train_deepchess(train, val)

## Evaluation

In [ ]:
deepchess = cb.model.get_deepchess()
deepchess.evaluate(train)

In [ ]:
deepchess.deepchess.summary()

## Engine

In [ ]:
engine = cb.engine.Engine(True)

## Play

In [ ]:
from IPython.display import SVG, display, clear_output
import chess

PLAY_WHITE = True

engine = cb.engine.Engine(not PLAY_WHITE)
player_turn = not engine.play_white

while not engine.board.is_game_over():
    display(SVG(chess.svg.board(engine.board, orientation=not engine.play_white, size=500)))

    if player_turn:
        move = chess.Move.from_uci(input("Enter a move: "))
    else:
        print("Computer calculating...")
        move = engine.calculate_move()

    clear_output(wait=False)

    if not engine.board.is_legal(move):
        continue

    player_turn = not player_turn
    engine.board.push(move)

display(SVG(chess.svg.board(engine.board, orientation=not engine.play_white, size=500)))
engine.board.outcome()

## Sandbox

In [ ]:
board = chess.Board()
print(board.pieces(chess.WHITE, chess.PAWN).tolist())

In [ ]:
train, val = cb.dataset.deepchess_dataset()

In [ ]:
deepchess = cb.model.get_deepchess()
deepchess.evaluate(train)

In [ ]:
deepchess.metrics

In [ ]:
# [1.2469837665557861, 0.6108400225639343]